# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [2]:
class Matrix:
    def __init__(self, n, m=None):
        if isinstance(n, list):  # Initialize from list of lists
            if not all(isinstance(row, list) for row in n):
                raise ValueError("Matrix must be initialized with a list of lists.")
            if not all(len(row) == len(n[0]) for row in n):
                raise ValueError("All rows must have the same length.")
            self.data = [row[:] for row in n]
        elif isinstance(n, int) and isinstance(m, int):  # Initialize from dimensions
            self.data = [[0] * m for _ in range(n)]
        else:
            raise ValueError("Invalid initialization parameters.")
    
    def shape(self):
        return len(self.data), len(self.data[0])
    
    def transpose(self):
        transposed = [[self.data[j][i] for j in range(len(self.data))] for i in range(len(self.data[0]))]
        return Matrix(transposed)
    
    def row(self, n):
        return Matrix([self.data[n][:]])
    
    def column(self, n):
        return Matrix([[row[n]] for row in self.data])
    
    def to_list(self):
        return [row[:] for row in self.data]
    
    def block(self, n_0, n_1, m_0, m_1):
        return Matrix([row[m_0:m_1] for row in self.data[n_0:n_1]])
    
    def __getitem__(self, index):
        if isinstance(index, tuple):
            i, j = index
            return self.data[i][j]
        return self.data[index]
    
    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            i, j = index
            self.data[i][j] = value
        else:
            self.data[index] = value
    
    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return self.scalarmul(other)
        elif isinstance(other, Matrix):
            return self.mat_mult(other)
        else:
            raise ValueError("Unsupported operand type for multiplication.")
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __add__(self, other):
        return self.add(other)
    
    def __sub__(self, other):
        return self.sub(other)
    
    def __eq__(self, other):
        return self.equals(other)
    
    def __repr__(self):
        return f"Matrix({self.to_list()})"
    
    def scalarmul(self, c):
        return Matrix([[c * elem for elem in row] for row in self.data])
    
    def add(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for addition.")
        return Matrix([[self.data[i][j] + N[i, j] for j in range(len(self.data[0]))] for i in range(len(self.data))])
    
    def sub(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for subtraction.")
        return Matrix([[self.data[i][j] - N[i, j] for j in range(len(self.data[0]))] for i in range(len(self.data))])
    
    def mat_mult(self, N):
        if self.shape()[1] != N.shape()[0]:
            raise ValueError("Matrix dimensions must be compatible for multiplication.")
        return Matrix([[sum(self.data[i][k] * N[k, j] for k in range(self.shape()[1])) for j in range(N.shape()[1])] for i in range(self.shape()[0])])
    
    def element_mult(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrix dimensions must match for element-wise multiplication.")
        return Matrix([[self.data[i][j] * N[i, j] for j in range(len(self.data[0]))] for i in range(len(self.data))])
    
    def equals(self, N):
        return self.shape() == N.shape() and all(self.data[i][j] == N[i, j] for i in range(len(self.data)) for j in range(len(self.data[0])))
    
# Standalone functions
def constant(n, m, c):
    return Matrix([[float(c)] * m for _ in range(n)])

def zeros(n, m):
    return constant(n, m, 0)

def ones(n, m):
    return constant(n, m, 1)

def eye(n):
    return Matrix([[1 if i == j else 0 for j in range(n)] for i in range(n)])


# Demonstration
def demonstrate_matrix_properties():
    A = Matrix([[1, 2], [3, 4]])
    B = Matrix([[2, 0], [1, 3]])
    C = Matrix([[0, 1], [2, 1]])
    I = eye(2)
    
    print("(AB)C = A(BC):", (A * B) * C == A * (B * C))
    print("A(B+C) = AB + AC:", A * (B + C) == A * B + A * C)
    print("AB ≠ BA:", A * B != B * A)
    print("AI = A:", A * I == A)

demonstrate_matrix_properties()


(AB)C = A(BC): True
A(B+C) = AB + AC: True
AB ≠ BA: True
AI = A: True
